# Meta Edge Prediction


Virtual envorinemnt
source .venv/bin/activate





Final project for Biological Networks, CSCI 3352

Be a lil sloppy that's ok. Get it done first, refine, comment, cleanup, later

Things we might wanna do:
- Dig into the data more, see why diff models work better than others


### Summary of Project

Write this at the end as our project may change as we go... 

Don't forget to fill in the comments in each of the sections!

Have instrucitons for importing the data and stuff



### Citations

Refine this aswell....

@article{ghasemian2020stacking,
  title = {Stacking models for nearly optimal link prediction in complex networks},
  author = {Ghasemian, Amir and Hosseinmardi, Homa and Galstyan, Aram and Airoldi, Edoardo M and Clauset, Aaron},
  journal = {Proceedings of the National Academy of Sciences},
  volume = {117},
  number = {38},
  pages = {23393--23400},
  year = {2020},
  publisher = {National Acad Sciences},
}

In [1]:
print(69)

69


In [2]:
import networkx as nx
from networkx.algorithms.link_prediction import jaccard_coefficient

import numpy as np
import matplotlib
import matplotlib.pylab as plt
import pickle
%matplotlib inline

from sklearn.model_selection import train_test_split

from concurrent.futures import ProcessPoolExecutor

import copy

# Importing Data

In [3]:
import pickle  

# Load the data
infile = open('./01-network-data/OLP_updated.pickle','rb')
df = pickle.load(infile)

# Filter df to get biological networks with > 200 nodes
df_bio_200_nodes = df[(df['networkDomain'] == 'Biological') & (df['number_nodes'] > 200)]

####### Convert the data (dataframe format) into networ-x graphs #######

# Only grab graphs from these subDomains
subDomains = ['Metabolic', 'Protein interactions', 'Tissue']

# Place to store the nx-graphs of each subDomain
Gs_subDomains = [] 

for subDomain in subDomains:
    
    # Isolate df-graphs of this subDomain
    df_subDomain = df_bio_200_nodes[(df_bio_200_nodes['subDomain'] == subDomain)]
    
    Gs_subDomain = [] # Place to store nx-graphs of a subDomain
    
    # On each df-graph, convert it to a nx-graph and store
    for i in range(len(df_subDomain)):
        
        # Get list-edges from df-graph
        df_subDomain_single_G = df_subDomain['edges_id'].iloc[i] 
        
        # Place to store tuple-edges
        G_tuple_list = [] 
        
        # Convert df edges into tuples
        for x in df_subDomain_single_G:
            G_tuple_list.append(tuple(x))
            
        # Convert the tuple list to a nx-graph
        G_nx = nx.from_edgelist(G_tuple_list)
            
        # Add the nx-graph to Gs_metabolic
        Gs_subDomain.append(G_nx) 
        
    Gs_subDomains.append(Gs_subDomain)


In [4]:
# Print stats on the data


# # MGD function from homework 4
# def compute_MGD(G):
#     # input : a networkx graph G
#     # output: the mean geodesic path length (defined in Lecture 2)

#     # YOUR CODE HERE
    
#     shortest_paths_data = list(nx.all_pairs_shortest_path_length(G)) # Get data of all shortest paths
#     shortest_paths = []
#     for i in shortest_paths_data: # Iterate through each node in the array/dict data
#         for j in i[1].values(): # Isolate the geodesics for this node
#             if j != 0 and j != np.inf: # Ensure geodesic is not 0 or infinity
#                 shortest_paths.append(j)
                
#     if len(shortest_paths) == 0: # If empty graph, return geodesic dist of inf.
#         return float('inf') # It needs to returns somehting if no edges, or else we get a divide by zero error
#     else: 
#         MGD = sum(shortest_paths) / len(shortest_paths) # Calculate MGD from shortest paths
#         return MGD







'''
for i in range(len(Gs_subDomains)): # On each subDomain print...
    
    Gs = Gs_subDomains[i] # Graphs of current subDomain
    
    print('Network subDomain:', subDomains[i])
    
    graph_count = len(Gs)
    print('# of graphs:', graph_count)
    
    avg_n = np.mean([len(G.nodes) for G in Gs])
    print(f'Avg number of nodes, n  = {avg_n}')
    
    avg_m = np.mean([len(G.edges) for G in Gs])
    print(f'Avg number of edges, m  = {avg_m}')

    kmeans = []
    for G in Gs:
        kmeans.append(2 * G.number_of_edges() / G.number_of_nodes())
    avg_kmeans = np.mean(kmeans)
    print(f'Avg mean degree,    <k> = %5.2f' % avg_kmeans)
    
    avg_C = np.mean([nx.transitivity(G) for G in Gs])
    print(f'Avg clustering coefficient, C     = %5.2f' % avg_C)
    
    # avg_ellmean = np.mean([compute_MGD(G) for G in Gs]) 
    # print(f'Avg mean geodesic distance, <ell> = %5.2f' % avg_ellmean)
    
    print('Sample rediculograms of subDomain:', subDomains[i])
    g1, g2 = np.random.choice(range(graph_count), 2, replace=False) # Randomly pick 2 graphs to display    
    nx.draw_spring(Gs[g1], node_size=10) # Rediculogram of a graph
    plt.show()
    nx.draw_spring(Gs[g2], node_size=10) # Rediculogram of a graph
    plt.show()
    
    print('\n') # Space out different subDomains
    
    
'''

"\nfor i in range(len(Gs_subDomains)): # On each subDomain print...\n    \n    Gs = Gs_subDomains[i] # Graphs of current subDomain\n    \n    print('Network subDomain:', subDomains[i])\n    \n    graph_count = len(Gs)\n    print('# of graphs:', graph_count)\n    \n    avg_n = np.mean([len(G.nodes) for G in Gs])\n    print(f'Avg number of nodes, n  = {avg_n}')\n    \n    avg_m = np.mean([len(G.edges) for G in Gs])\n    print(f'Avg number of edges, m  = {avg_m}')\n\n    kmeans = []\n    for G in Gs:\n        kmeans.append(2 * G.number_of_edges() / G.number_of_nodes())\n    avg_kmeans = np.mean(kmeans)\n    print(f'Avg mean degree,    <k> = %5.2f' % avg_kmeans)\n    \n    avg_C = np.mean([nx.transitivity(G) for G in Gs])\n    print(f'Avg clustering coefficient, C     = %5.2f' % avg_C)\n    \n    # avg_ellmean = np.mean([compute_MGD(G) for G in Gs]) \n    # print(f'Avg mean geodesic distance, <ell> = %5.2f' % avg_ellmean)\n    \n    print('Sample rediculograms of subDomain:', subDomains[i]

# Run Edge Prediction on Graphs

Some cool description stuff abt this section maybe idk

In [5]:
# Mage G-Observerd and G-Missing

# !!!!!!!!!!!
# To start, just do everything on the subDomain: metabolic
Gs = Gs_subDomains[0]

seed = 630
alpha = 0.8 # Number of edges to observe

# IMPORTANT!!!
# Now instead of being a graph, the nodes and edges will be represented as a list
edges_o = [] # G-Observed
edges_missing = [] # G-Missing
nodes = []

for G in Gs:
    G_nodes = []
    for node in G.nodes():
        G_nodes.append(node)
    nodes.append(G_nodes)
    
    edge_o = []
    edge_missing = []
    for edge in G.edges():
        if alpha > np.random.rand():
            edge_o.append(edge)
        else:
            edge_missing.append(edge)
    edges_o.append(edge_o)
    edges_missing.append(edge_missing)

    
    
    
# Convert the edges into graphs
Gs_o = []
Gs_missing = []

for e_o, e_missing in zip(edges_o, edges_missing):
    
    # Create the graphs
    G_o = nx.Graph()
    G_missing = nx.Graph()
    
    # Add edges to graphs
    G_o.add_edges_from(e_o)
    G_missing.add_edges_from(e_missing)
    
    # Add graphs to array of graphs
    Gs_o.append(G_o)
    Gs_missing.append(G_missing)
    
    
    
    
# Sanity check
g = 3
print(Gs[g])
print('Edges observed', len(edges_o[g]))
print('Edges missing', len(edges_missing[g]))
print("Example edge from edges_o:", edges_o[g][4])

print('\n')

print('G missing:', Gs_missing[g])

Graph with 619 nodes and 1501 edges
Edges observed 1196
Edges missing 305
Example edge from edges_o: (0, 52)


G missing: Graph with 312 nodes and 305 edges


# Get Lists of edge/nonedge Data

This data tells for each potential new edge, if it was actually a nonedge, it will have a 0, if it was actually an edge, it will have a 1

In [6]:
def create_edgenonedge_data(G_o, G_missing):
    
    data = []
    
    for u, v in nx.non_edges(G_o):
        if G_missing.has_edge(u,v):
            data.append(1)
        else:
            data.append(0)
        
    # Convert to NumPy array and reshape to (N, 1)
    return np.array(data).reshape(-1, 1)

edgenonedge_data = [] # Place to store predictions

for G_o, G_missing in zip(Gs_o, Gs_missing): # Create edgenonedge_data on each graph
    d = create_edgenonedge_data(G_o, G_missing)
    edgenonedge_data.append(d)

print(edgenonedge_data[0].shape)

(363242, 1)


In [7]:
# Save data for later use

# Save data 
with open('04-edgenonedge-data/edgenonedge_data.pkl', 'wb') as f:
    pickle.dump(edgenonedge_data, f)


## Run predictors on the graphs

In [8]:
# Run jaccard edge predictor on graphs

# Function to compute edge predictions on all edges in a single graph
def predictor_jaccard(G_o):

    # Compute predictor scores
    predictions = dict(((u, v), p) for u, v, p in nx.jaccard_coefficient(G_o))
    
    ordered_predictions = []

    # Grab predictor scores in the same order as the other predictor functions
    for u, v in nx.non_edges(G_o):
        p = predictions.get((u, v))
        ordered_predictions.append(p)
        
    # Convert to NumPy array and reshape to (N, 1)
    return np.array(ordered_predictions).reshape(-1, 1)




jacc_edge_scores = [] # Place to store predictions

for G_o in Gs_o: # Make predictions on each graph
    p = predictor_jaccard(G_o)
    jacc_edge_scores.append(p)



    
jacc_edge_scores[0].shape

(363242, 1)

In [9]:
# Run adamic-adar edge predictor on graphs

# Function to compute edge predictions on all edges in a single graph
def predictor_adamic_adar(G_o):

    # Compute predictor scores
    predictions = dict(((u, v), p) for u, v, p in nx.adamic_adar_index(G_o))
    
    ordered_predictions = []

    # Grab predictor scores in the same order as the other predictor functions
    for u, v in nx.non_edges(G_o):
        p = predictions.get((u, v))
        ordered_predictions.append(p)
        
    # Convert to NumPy array and reshape to (N, 1)
    return np.array(ordered_predictions).reshape(-1, 1)


adamic_adar_edge_scores = [] # Place to store predictions

for G_o in Gs_o: # Make predictions on each graph
    p = predictor_adamic_adar(G_o)
    adamic_adar_edge_scores.append(p)

'''
After switching to VS Code from Jupyter Notebook to speed up my
workflow, parallel proscessing started throwing tons of errors. So
I will be switching to a for loop. On VS Code the for loop actually
Performs about the same as the parallel proscessing.
'''
# # Use parallel proscessing to speed up calculations!
# with ProcessPoolExecutor() as executor: # Use max number of executors
#     jacc_edge_scores = list(
#         executor.map(predictor_jaccard, Gs_o) # Pass each graph's edges into the predictor
#     )

'\nAfter switching to VS Code from Jupyter Notebook to speed up my\nworkflow, parallel proscessing started throwing tons of errors. So\nI will be switching to a for loop. On VS Code the for loop actually\nPerforms about the same as the parallel proscessing.\n'

In [10]:
# Run geodesic edge predictor on graphs

# Function to compute edge predictions on all edges in a single graph
def predictor_geodesic(G_o):
    
    predictions = []
    
    for u, v in nx.non_edges(G_o):
        try:
            score = nx.shortest_path_length(G_o, u, v)
            predictions.append(score)
        except nx.NetworkXNoPath: 
            # No path - set score to 0
            predictions.append(0)
        
    # Convert to NumPy array and reshape to (N, 1)
    return np.array(predictions).reshape(-1, 1)

geodesic_edge_scores = [] # Place to store predictions

for G_o in Gs_o: # Make predictions on each graph
    p = predictor_geodesic(G_o)
    geodesic_edge_scores.append(p)

In [11]:
# Run common-neighbors edge predictor on graphs

# Function to compute edge predictions on all edges in a single graph
def predictor_common_neighbors(G_o):
    
    predictions = []
    
    for u, v in nx.non_edges(G_o):
        score = len(list(nx.common_neighbors(G_o, u, v)))
        predictions.append(score)
        
    # Convert to NumPy array and reshape to (N, 1)
    return np.array(predictions).reshape(-1, 1)

common_neighbors_edge_scores = [] # Place to store predictions

for G_o in Gs_o: # Make predictions on each graph
    p = predictor_common_neighbors(G_o)
    common_neighbors_edge_scores.append(p)

In [12]:
# Run degree-product edge predictor on graphs

# Function to compute edge predictions on all edges in a single graph
def predictor_degree_prod(G_o):
    
    predictions = []
    
    for u, v in nx.non_edges(G_o):
        score = G_o.degree[u] * G_o.degree[v]
        predictions.append(score)
        
    # Convert to NumPy array and reshape to (N, 1)
    return np.array(predictions).reshape(-1, 1)

degree_prod_edge_scores = [] # Place to store predictions

for G_o in Gs_o: # Make predictions on each graph
    p = predictor_degree_prod(G_o)
    degree_prod_edge_scores.append(p)

In [13]:
# Predict missing edges with baseline predictor (random)

baseline_edge_scores = [] # Place to store predictions

for G in jacc_edge_scores:
    predictions = []
    for edge in G:
        predictions.append(np.random.uniform(0,1))
    baseline_edge_scores.append(np.array(predictions).reshape(-1, 1))
    
    
    

## Sanity Check!

Im scared lol

In [14]:

'''
# All predictior edge scores:

jacc_edge_scores
adamic_adar_edge_scores
geodesic_edge_scores
common_neighbors_edge_scores
degree_prod_edge_scores
baseline_edge_scores
'''

predictor_names = ['jacc_edge_scores', 'adamic_adar_edge_scores','geodesic_edge_scores',
                   'common_neighbors_edge_scores','degree_prod_edge_scores',
                   'baseline_edge_scores']

ps_grouped_together = [jacc_edge_scores, adamic_adar_edge_scores, geodesic_edge_scores,
                               common_neighbors_edge_scores, degree_prod_edge_scores,
                               baseline_edge_scores]

g = 5 # Graph index
e = 420 # Edge index


for i in range(len(predictor_names)):
    print(f'Predictor:   {predictor_names[i]}   sameple score Edge Score:   {ps_grouped_together[i][g][e]}')

print('\n')

for i in range(len(predictor_names)):
    print(f'Predictor:   {predictor_names[i]}   shape:   {ps_grouped_together[i][0].shape}')

Predictor:   jacc_edge_scores   sameple score Edge Score:   [0.]
Predictor:   adamic_adar_edge_scores   sameple score Edge Score:   [0.]
Predictor:   geodesic_edge_scores   sameple score Edge Score:   [3]
Predictor:   common_neighbors_edge_scores   sameple score Edge Score:   [0]
Predictor:   degree_prod_edge_scores   sameple score Edge Score:   [155]
Predictor:   baseline_edge_scores   sameple score Edge Score:   [0.31213935]


Predictor:   jacc_edge_scores   shape:   (363242, 1)
Predictor:   adamic_adar_edge_scores   shape:   (363242, 1)
Predictor:   geodesic_edge_scores   shape:   (363242, 1)
Predictor:   common_neighbors_edge_scores   shape:   (363242, 1)
Predictor:   degree_prod_edge_scores   shape:   (363242, 1)
Predictor:   baseline_edge_scores   shape:   (363242, 1)


## Save The Predictions

Whew those predictions take forever to calculate. Lets save them so we don't have to run this code everytime!

In [15]:
predictors = copy.deepcopy([jacc_edge_scores, adamic_adar_edge_scores, geodesic_edge_scores, 
              common_neighbors_edge_scores, degree_prod_edge_scores, 
             baseline_edge_scores])

# Save data 
with open('02-predictor-data/raw/predictors.pkl', 'wb') as f:
    pickle.dump(predictors, f)


# Get edge metadata

In [16]:
import networkx as nx
import numpy as np


# Takes my laptop ~40 sec to run

def create_edge_metadata(G_o, G_p):
    data = []
    
    # Precompute graph-level metadata
    node_count = G_o.number_of_nodes()
    edge_count = G_o.number_of_edges()
    density = nx.density(G_o)
    avg_degree = sum(dict(G_o.degree()).values()) / node_count
    clustering_coeff = nx.average_clustering(G_o)
    jacc_edge_scores = G_p[0]
    adamic_adar_edge_scores = G_p[1]
    geodesic_edge_scores = G_p[2]
    common_neighbors_edge_scores = G_p[3]
    degree_prod_edge_scores = G_p[4]
    
    try:
        diameter = nx.diameter(G_o) if nx.is_connected(G_o) else -1  # Use -1 for disconnected graphs
    except nx.NetworkXError:
        diameter = -1  # Handle cases where diameter cannot be computed
    
    num_connected_components = nx.number_connected_components(G_o)
    
    # Precompute node-level metadata
    degrees = dict(G_o.degree())
    pagerank = nx.pagerank(G_o)
    
    # Iterate over non-edges and compute edge metadata
    i = 0
    for u, v in nx.non_edges(G_o):  # Order maintained by nx.non_edges()
        if i%100000 == 0:
            print('Completed the', i,'th iteration')
        # Edge-specific metadata
        degree_u = degrees.get(u, 0)
        degree_v = degrees.get(v, 0)
        pagerank_u = pagerank.get(u, 0)
        pagerank_v = pagerank.get(v, 0)
        
        # Combine edge and graph metadata
        edge_entry = [
            degree_u, degree_v,       # Node degrees
            pagerank_u, pagerank_v,   # Node PageRank
            node_count, edge_count,   # Graph size
            density, avg_degree,      # Graph density and avg degree
            diameter,                 # Graph diameter
            num_connected_components, # Number of connected components
            clustering_coeff,         # Avg clustering coefficient
            jacc_edge_scores[i].item(), adamic_adar_edge_scores[i].item(),
            geodesic_edge_scores[i].item(), common_neighbors_edge_scores[i].item(),
            degree_prod_edge_scores[i].item()

        ]
        data.append(edge_entry)
        i += 1

    # Convert to NumPy array
    return np.array(data)

# Example usage:
edge_metadata = []  # Place to store metadata arrays

for i in range(len(Gs_o)):  # Create edge metadata for each graph in Gs_o
    G_o = Gs_o[i]
    G_p = [] # Place to store predictor data for this graph
    for p in predictors: # Over each predictor
        G_p.append(p[i]) # Get this graphs predictor data
    d = create_edge_metadata(G_o, G_p)
    print('Sample predictor score for a graph:', G_p[0][0].item())

    edge_metadata.append(d)

Completed the 0 th iteration
Completed the 100000 th iteration
Completed the 200000 th iteration
Completed the 300000 th iteration
Sample predictor score for a graph: 0.006172839506172839
Completed the 0 th iteration
Completed the 100000 th iteration
Completed the 200000 th iteration
Completed the 300000 th iteration
Sample predictor score for a graph: 0.0
Completed the 0 th iteration
Completed the 100000 th iteration
Completed the 200000 th iteration
Sample predictor score for a graph: 0.06060606060606061
Completed the 0 th iteration
Completed the 100000 th iteration
Sample predictor score for a graph: 0.0
Completed the 0 th iteration
Sample predictor score for a graph: 0.013513513513513514
Completed the 0 th iteration
Completed the 100000 th iteration
Completed the 200000 th iteration
Completed the 300000 th iteration
Sample predictor score for a graph: 0.006369426751592357
Completed the 0 th iteration
Completed the 100000 th iteration
Sample predictor score for a graph: 0.0
Complete

In [17]:
# Print shape of metadata for the first graph
print(edge_metadata[0].shape)

# Example for inspecting edge_metadata
graph_index = 0  # Change this to inspect other graphs

# Check shape of edge_metadata for the selected graph
print(f"edge_metadata[{graph_index}] shape:", edge_metadata[graph_index].shape)

# Check for constant or uniform data within the selected graph
print(f"Unique rows in edge_metadata[{graph_index}]:", np.unique(edge_metadata[graph_index], axis=0).shape[0])

# Inspect a few rows of the selected graph's edge metadata
print(f"First 5 rows of edge_metadata[{graph_index}]:\n", edge_metadata[graph_index][:5])

(363242, 16)
edge_metadata[0] shape: (363242, 16)
Unique rows in edge_metadata[0]: 259893
First 5 rows of edge_metadata[0]:
 [[ 1.61000000e+02  2.00000000e+00  4.00512687e-02  6.20650058e-04
   8.55000000e+02  1.84300000e+03  5.04813947e-03  4.31111111e+00
  -1.00000000e+00  4.00000000e+00  1.08891926e-01  6.17283951e-03
   2.41363138e-01  2.00000000e+00  1.00000000e+00  3.22000000e+02]
 [ 1.61000000e+02  3.00000000e+00  4.00512687e-02  7.79806973e-04
   8.55000000e+02  1.84300000e+03  5.04813947e-03  4.31111111e+00
  -1.00000000e+00  4.00000000e+00  1.08891926e-01  6.13496933e-03
   2.19593112e-01  2.00000000e+00  1.00000000e+00  4.83000000e+02]
 [ 1.61000000e+02  4.00000000e+00  4.00512687e-02  9.72611570e-04
   8.55000000e+02  1.84300000e+03  5.04813947e-03  4.31111111e+00
  -1.00000000e+00  4.00000000e+00  1.08891926e-01  6.09756098e-03
   2.71085031e-01  2.00000000e+00  1.00000000e+00  6.44000000e+02]
 [ 1.61000000e+02  2.00000000e+00  4.00512687e-02  5.81520716e-04
   8.55000000e

In [18]:
# Save data for later use

# Takes ~

# Save data 
with open('05-edge-metadata/edge_metadata.pkl', 'wb') as f:
    pickle.dump(edge_metadata, f)

In [19]:
print(edge_metadata[0][0][10])
print(edge_metadata[0][0][11])

0.10889192561310057
0.006172839506172839


In [20]:
# Function to validate metadata for specific edges
def validate_metadata(G_o, metadata, predictor_scores, num_edges=5):
    """
    Validate metadata and predictor scores for the first and last few edges.
    
    Args:
        G_o (Graph): The original graph.
        metadata (np.array): The computed metadata for edges.
        predictor_scores (np.array): Predictor scores (e.g., Jaccard) for edges.
        num_edges (int): Number of edges to check from the start and end.
    """
    # Get the list of non-edges
    non_edges = list(nx.non_edges(G_o))
    
    # Check first few edges
    print("Checking first few edges...")
    for i, (u, v) in enumerate(non_edges[:num_edges]):
        print(f"Edge ({u}, {v}):")
        print(f"  Metadata: {metadata[i]}")
        print(f"  Jaccard score: {predictor_scores[i][0]}")  # Assuming predictor_scores is reshaped (-1, 1)
        # Optional: Compute metadata on-the-fly and compare
        degree_u = G_o.degree(u)
        degree_v = G_o.degree(v)
        print(f"  Computed Degree_u: {degree_u}, Degree_v: {degree_v}")
        print("---")

    # Check last few edges
    print("\nChecking last few edges...")
    for i, (u, v) in enumerate(non_edges[-num_edges:]):
        idx = len(non_edges) - num_edges + i  # Adjust index for metadata and predictors
        print(f"Edge ({u}, {v}):")
        print(f"  Metadata: {metadata[idx]}")
        print(f"  Jaccard score: {predictor_scores[idx][0]}")
        # Optional: Compute metadata on-the-fly and compare
        degree_u = G_o.degree(u)
        degree_v = G_o.degree(v)
        print(f"  Computed Degree_u: {degree_u}, Degree_v: {degree_v}")
        print("---")


# Example Usage
graph_index = 0  # Choose the graph index to test
G_o = Gs_o[graph_index]
metadata = edge_metadata[graph_index]
predictor_scores = jacc_edge_scores[graph_index]

# Validate metadata for the first and last few edges
validate_metadata(G_o, metadata, predictor_scores, num_edges=5)

Checking first few edges...
Edge (0, 2):
  Metadata: [ 1.61000000e+02  2.00000000e+00  4.00512687e-02  6.20650058e-04
  8.55000000e+02  1.84300000e+03  5.04813947e-03  4.31111111e+00
 -1.00000000e+00  4.00000000e+00  1.08891926e-01  6.17283951e-03
  2.41363138e-01  2.00000000e+00  1.00000000e+00  3.22000000e+02]
  Jaccard score: 0.006172839506172839
  Computed Degree_u: 161, Degree_v: 2
---
Edge (0, 3):
  Metadata: [ 1.61000000e+02  3.00000000e+00  4.00512687e-02  7.79806973e-04
  8.55000000e+02  1.84300000e+03  5.04813947e-03  4.31111111e+00
 -1.00000000e+00  4.00000000e+00  1.08891926e-01  6.13496933e-03
  2.19593112e-01  2.00000000e+00  1.00000000e+00  4.83000000e+02]
  Jaccard score: 0.006134969325153374
  Computed Degree_u: 161, Degree_v: 3
---
Edge (0, 4):
  Metadata: [ 1.61000000e+02  4.00000000e+00  4.00512687e-02  9.72611570e-04
  8.55000000e+02  1.84300000e+03  5.04813947e-03  4.31111111e+00
 -1.00000000e+00  4.00000000e+00  1.08891926e-01  6.09756098e-03
  2.71085031e-01  2.

In [21]:
def validate_metadata_comparisons(G_o, G_s, metadata, predictor_scores, num_edges=3):
    """
    Compare Jaccard score, Degree_u, and Degree_v for observed graph (G_o),
    freshly computed values, and original graph (G_s).

    Args:
        G_o (Graph): The observed graph.
        G_s (Graph): The original graph.
        metadata (np.array): The computed metadata for edges in G_o.
        predictor_scores (np.array): Predictor scores (e.g., Jaccard) for edges in G_o.
        num_edges (int): Number of edges to check from the start and end.
    """
    # Get the list of non-edges in G_o and edges in G_s
    non_edges = list(nx.non_edges(G_o))
    original_edges = list(G_s.edges())
    
    def compute_metadata(G, u, v):
        """
        Compute metadata stats from scratch for a given edge (u, v) in graph G.
        """
        degree_u = G.degree(u)
        degree_v = G.degree(v)
        jaccard = list(nx.jaccard_coefficient(G, [(u, v)]))[0][2] if G.has_edge(u, v) or not nx.is_directed(G) else 0
        return {
            "Jaccard": round(jaccard, 6),
            "Degree_u": round(degree_u, 6),
            "Degree_v": round(degree_v, 6)
        }
    
    # Check first few edges
    print("Comparing first few edges...")
    for i, (u, v) in enumerate(non_edges[:num_edges]):
        print(f"Edge ({u}, {v}):")
        
        # Pipeline metadata
        print(f"  Pipeline Metadata (G_o):")
        print(f"    Jaccard score: {round(predictor_scores[i][0], 6)}")
        print(f"    Degree_u: {metadata[i][0]:.6f}, Degree_v: {metadata[i][1]:.6f}")
        
        # Freshly computed metadata for G_o
        fresh_meta = compute_metadata(G_o, u, v)
        print(f"  Freshly Computed Metadata (G_o): {fresh_meta}")
        
        # Freshly computed metadata for G_s
        orig_meta = compute_metadata(G_s, u, v)
        print(f"  Metadata from Original Graph (G_s): {orig_meta}")
        print("---")
    
    # Check last few edges
    print("\nComparing last few edges...")
    for i, (u, v) in enumerate(non_edges[-num_edges:]):
        idx = len(non_edges) - num_edges + i
        print(f"Edge ({u}, {v}):")
        
        # Pipeline metadata
        print(f"  Pipeline Metadata (G_o):")
        print(f"    Jaccard score: {round(predictor_scores[idx][0], 6)}")
        print(f"    Degree_u: {metadata[idx][0]:.6f}, Degree_v: {metadata[idx][1]:.6f}")
        
        # Freshly computed metadata for G_o
        fresh_meta = compute_metadata(G_o, u, v)
        print(f"  Freshly Computed Metadata (G_o): {fresh_meta}")
        
        # Freshly computed metadata for G_s
        orig_meta = compute_metadata(G_s, u, v)
        print(f"  Metadata from Original Graph (G_s): {orig_meta}")
        print("---")


# Example Usage
graph_index = 0  # Test the first graph
G_o = Gs_o[graph_index]  # Observed graph
G_s = Gs[graph_index]    # Original graph
metadata = edge_metadata[graph_index]
predictor_scores = jacc_edge_scores[graph_index]

# Validate and compare metadata
validate_metadata_comparisons(G_o, G_s, metadata, predictor_scores, num_edges=3)

Comparing first few edges...
Edge (0, 2):
  Pipeline Metadata (G_o):
    Jaccard score: 0.006173
    Degree_u: 161.000000, Degree_v: 2.000000
  Freshly Computed Metadata (G_o): {'Jaccard': 0.006173, 'Degree_u': 161, 'Degree_v': 2}
  Metadata from Original Graph (G_s): {'Jaccard': 0.005291, 'Degree_u': 188, 'Degree_v': 2}
---
Edge (0, 3):
  Pipeline Metadata (G_o):
    Jaccard score: 0.006135
    Degree_u: 161.000000, Degree_v: 3.000000
  Freshly Computed Metadata (G_o): {'Jaccard': 0.006135, 'Degree_u': 161, 'Degree_v': 3}
  Metadata from Original Graph (G_s): {'Jaccard': 0.010582, 'Degree_u': 188, 'Degree_v': 3}
---
Edge (0, 4):
  Pipeline Metadata (G_o):
    Jaccard score: 0.006098
    Degree_u: 161.000000, Degree_v: 4.000000
  Freshly Computed Metadata (G_o): {'Jaccard': 0.006098, 'Degree_u': 161, 'Degree_v': 4}
  Metadata from Original Graph (G_s): {'Jaccard': 0.010471, 'Degree_u': 188, 'Degree_v': 5}
---

Comparing last few edges...
Edge (877, 878):
  Pipeline Metadata (G_o):
    